<a href="https://colab.research.google.com/github/ssook1222/bigdata-analysist-certificate/blob/master/bigdata-analysist-certificate/task2/on_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 정시 배송 여부 판단

### 라이브러리 다 때려박기

In [35]:
import pandas as pd
import numpy as np
import sklearn as sk

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
import warnings
warnings.filterwarnings('ignore')

### 데이터 불러오기

In [40]:
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv")
y_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv")

In [4]:
display(x_train)
display(y_train)
display(x_test)

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833
...,...,...,...,...,...,...,...,...,...,...,...
6593,8610,F,Road,5,2,221,6,medium,M,4,1952
6594,3390,F,Ship,4,5,256,3,medium,M,10,4504
6595,5150,F,Ship,3,1,217,4,medium,F,1,5761
6596,3950,F,Road,4,5,174,3,medium,F,8,5576


,ID,Reached.on.Time_Y.N
0,6045,0
1,44,1
2,7940,1
3,1596,1
4,4395,1
...,...,...
6593,8610,1
6594,3390,1
6595,5150,1
6596,3950,0


,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6811,D,Ship,5,2,259,5,low,F,7,1032
1,4320,F,Ship,3,5,133,3,medium,F,4,5902
2,5732,F,Road,3,4,191,5,medium,F,4,4243
3,7429,D,Ship,4,2,221,3,low,M,10,4126
4,2191,D,Flight,4,5,230,2,low,F,38,2890
...,...,...,...,...,...,...,...,...,...,...,...
4396,2610,F,Flight,4,1,157,3,medium,M,31,1712
4397,3406,B,Road,3,5,139,2,medium,M,7,5536
4398,10395,A,Road,4,1,170,10,medium,F,3,5211
4399,3646,B,Ship,3,1,244,3,medium,F,1,5695


### 데이터 전처리

####  결측치 개수 확인

결측치 제거할 필요가 없음.

In [5]:
x_train[(x_train.isna() == True)].count()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

#### 레이블 인코딩 진행

object type을 전부 int로

In [6]:
le = LabelEncoder()

x_train['Warehouse_block'] = le.fit_transform(x_train['Warehouse_block'])
x_train['Mode_of_Shipment'] = le.fit_transform(x_train['Mode_of_Shipment'])
x_train['Product_importance'] = le.fit_transform(x_train['Product_importance'])
x_train['Customer_care_calls'] = le.fit_transform(x_train['Customer_care_calls'])
x_train['Gender'] = le.fit_transform(x_train['Gender'])

x_test['Warehouse_block'] = le.fit_transform(x_test['Warehouse_block'])
x_test['Mode_of_Shipment'] = le.fit_transform(x_test['Mode_of_Shipment'])
x_test['Product_importance'] = le.fit_transform(x_test['Product_importance'])
x_test['Customer_care_calls'] = le.fit_transform(x_test['Customer_care_calls'])
x_test['Gender'] = le.fit_transform(x_test['Gender'])

### 하이퍼 파라미터 튜닝

In [10]:
rf = RandomForestClassifier()

In [27]:
param = {
    'n_estimators' : [10],
    'max_depth' : [6, 8],
    'min_samples_leaf' : [8, 12],
    'min_samples_split' : [8, 16]
}

In [13]:
rf = RandomForestClassifier(random_state = 0, n_jobs = -1)

In [28]:
grid_cv = GridSearchCV(rf, param_grid = param, cv = 3, n_jobs = -1, refit = True)

In [29]:
grid_cv.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
             n_jobs=-1,
             param_grid={'max_depth': [6, 8], 'min_samples_leaf': [8, 12],
                         'min_samples_split': [8, 16], 'n_estimators': [10]})

In [31]:
print('최적 하이퍼 파라미터: ', grid_cv.best_params_)

최적 하이퍼 파라미터:  {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 10}


### 모델링 진행

In [32]:
rf = RandomForestClassifier(random_state = 0, n_jobs = -1, n_estimators=10, max_depth = 6, min_samples_leaf=8, min_samples_split=8)

In [33]:
rf_raw = rf.fit(x_train, y_train)

In [37]:
pred = rf_raw.predict(x_test)

In [43]:
pred

array([[7425,    0],
       [7015,    0],
       [5153,    0],
       ...,
       [6522,    0],
       [3598,    0],
       [1209,    1]])

In [44]:
print('예측 정확도: {:.4f}'.format(accuracy_score(y_test['Reached.on.Time_Y.N'], pred[:,1])))

예측 정확도: 0.6805


### 제출용 데이터 만들어서 내기

In [45]:
x_test

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6811,D,Ship,5,2,259,5,low,F,7,1032
1,4320,F,Ship,3,5,133,3,medium,F,4,5902
2,5732,F,Road,3,4,191,5,medium,F,4,4243
3,7429,D,Ship,4,2,221,3,low,M,10,4126
4,2191,D,Flight,4,5,230,2,low,F,38,2890
...,...,...,...,...,...,...,...,...,...,...,...
4396,2610,F,Flight,4,1,157,3,medium,M,31,1712
4397,3406,B,Road,3,5,139,2,medium,M,7,5536
4398,10395,A,Road,4,1,170,10,medium,F,3,5211
4399,3646,B,Ship,3,1,244,3,medium,F,1,5695


In [46]:
submit = pd.DataFrame({"ID": x_test['ID'], "on_time_y/n" :pred[:,1]})
print(submit.head(5))

     ID  on_time_y/n
0  6811            0
1  4320            0
2  5732            0
3  7429            0
4  2191            1


In [47]:
submit.to_csv("result.csv", index = False)